In [3]:
# Import all required libraries for image preprocessing
import os                          # For file system operations
import numpy as np                 # For numerical operations
import pandas as pd                # For data manipulation
import matplotlib.pyplot as plt    # For plotting visualizations
from PIL import Image, ImageEnhance  # For image loading and enhancement
import cv2                         # For advanced image processing
from sklearn.preprocessing import LabelEncoder  # For encoding class labels
from sklearn.model_selection import train_test_split  # For data splitting
import tensorflow as tf           # For deep learning operations
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # For augmentation
import random                     # For random operations
import warnings                   # To handle warnings
warnings.filterwarnings('ignore')  # Suppress warning messages

# Set random seed for reproducible results
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

print(" All libraries imported successfully")


 All libraries imported successfully


In [4]:
# Define the dataset path
dataset_path = '/kaggle/input/pre-process-data/For-pre-process'

# Display dataset path
print(" Dataset path:", dataset_path)

# Function to explore dataset structure
def explore_dataset():
    """Explore the dataset structure and count images in each category"""
    print("\n Exploring dataset structure...")
    
    # Get all category folders
    categories = []
    total_images = 0
    
    # Loop through each item in dataset directory
    for item in os.listdir(dataset_path):
        item_path = os.path.join(dataset_path, item)
        
        # Check if item is a directory (category folder)
        if os.path.isdir(item_path):
            # Count image files in category
            image_files = [f for f in os.listdir(item_path) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            image_count = len(image_files)
            
            categories.append(item)
            total_images += image_count
            print(f" {item}: {image_count} images")

    print(f"\n Total categories: {len(categories)}")
    print(f" Total images: {total_images}")
    print(f" Categories: {categories}")
    
    return categories, total_images

# Explore the dataset
categories, total_count = explore_dataset()


 Dataset path: /kaggle/input/pre-process-data/For-pre-process

 Exploring dataset structure...
 metal: 2000 images
 glass: 2000 images
 paper: 2000 images
 cardboard: 2000 images
 plastic: 2000 images

 Total categories: 5
 Total images: 10000
 Categories: ['metal', 'glass', 'paper', 'cardboard', 'plastic']


In [5]:
# Function to load and validate images
def load_image(image_path):
    """Load image from path and convert to RGB format"""
    try:
        # Open image using PIL
        image = Image.open(image_path)
        
        # Convert to RGB if image has different format
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # Check minimum size (filter very small images)
        if image.size[0] < 50 or image.size[1] < 50:
            return None
            
        return image
    except Exception as e:
        # Return None if image cannot be loaded
        print(f" Error loading {image_path}: {e}")
        return None

# Function to resize images to target size
def resize_image(image, target_size=(256, 256)):
    """Resize image to specified dimensions"""
    # Use LANCZOS resampling for better quality
    resized = image.resize(target_size, Image.Resampling.LANCZOS)
    return resized

# Function to convert PIL image to numpy array
def image_to_array(image):
    """Convert PIL image to numpy array"""
    # Convert PIL image to numpy array
    img_array = np.array(image)
    return img_array

# Function to normalize pixel values
def normalize_image(image_array):
    """Normalize pixel values from 0-255 to 0-1 range"""
    # Convert to float and divide by 255 to get 0-1 range
    normalized = image_array.astype(np.float32) / 255.0
    return normalized

print(" Image processing functions defined successfully")


 Image processing functions defined successfully


In [6]:
# Function to apply zoom augmentation
def apply_zoom(image, zoom_range=(0.8, 1.2)):
    """Apply random zoom in/out to image"""
    zoom_factor = random.uniform(zoom_range[0], zoom_range[1])
    
    # Get image dimensions
    width, height = image.size
    
    if zoom_factor < 1.0:
        # Zoom out - resize smaller and pad with white
        new_width = int(width * zoom_factor)
        new_height = int(height * zoom_factor)
        resized = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
        # Create white background and paste resized image in center
        zoomed = Image.new('RGB', (width, height), (255, 255, 255))
        x_offset = (width - new_width) // 2
        y_offset = (height - new_height) // 2
        zoomed.paste(resized, (x_offset, y_offset))
        return zoomed
    else:
        # Zoom in - resize larger and crop center
        new_width = int(width * zoom_factor)
        new_height = int(height * zoom_factor)
        resized = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
        # Crop center to original size
        x_offset = (new_width - width) // 2
        y_offset = (new_height - height) // 2
        return resized.crop((x_offset, y_offset, x_offset + width, y_offset + height))

# Function to apply rotation augmentation
def apply_rotation(image, angle_range=(-30, 30)):
    """Apply random rotation to image"""
    # Get random angle in specified range
    angle = random.uniform(angle_range[0], angle_range[1])
    # Rotate image with white background fill
    rotated = image.rotate(angle, expand=True, fillcolor=(255, 255, 255))
    return rotated

# Function to apply brightness augmentation
def apply_brightness(image, brightness_range=(0.7, 1.3)):
    """Apply random brightness adjustment"""
    # Create brightness enhancer
    enhancer = ImageEnhance.Brightness(image)
    # Get random brightness factor
    factor = random.uniform(brightness_range[0], brightness_range[1])
    # Apply brightness adjustment
    brightened = enhancer.enhance(factor)
    return brightened

# Function to apply random augmentation
def apply_augmentation(image, augment_type='random'):
    """Apply specified augmentation type to image"""
    if augment_type == 'random':
        # Randomly choose augmentation type
        aug_choice = random.choice(['zoom', 'rotate', 'brightness'])
    else:
        aug_choice = augment_type
    
    # Apply selected augmentation
    if aug_choice == 'zoom':
        augmented = apply_zoom(image)
    elif aug_choice == 'rotate':
        augmented = apply_rotation(image)
        # Resize after rotation to maintain target size
        augmented = resize_image(augmented)
    elif aug_choice == 'brightness':
        augmented = apply_brightness(image)
    else:
        augmented = image  # No augmentation
    
    return augmented

print(" Augmentation functions defined successfully")


 Augmentation functions defined successfully


In [7]:
# Function to create label encoder for categories
def create_label_encoder(categories):
    """Create and fit label encoder for garbage categories"""
    # Create label encoder instance
    label_encoder = LabelEncoder()
    
    # Fit encoder with category names
    label_encoder.fit(categories)
    
    # Display label mappings
    print(" Label Encoding Mappings:")
    for i, category in enumerate(categories):
        encoded_label = label_encoder.transform([category])[0]
        print(f"   {category} → {encoded_label}")
    
    return label_encoder

# Function to encode category name to numerical label
def encode_label(category, label_encoder):
    """Convert category name to numerical label"""
    encoded = label_encoder.transform([category])[0]
    return encoded

# Function to decode numerical label back to category name
def decode_label(encoded_label, label_encoder):
    """Convert numerical label back to category name"""
    decoded = label_encoder.inverse_transform([encoded_label])[0]
    return decoded

# Create label encoder for the dataset
if categories:
    label_encoder = create_label_encoder(categories)
    print(f"\n Label encoder created for {len(categories)} categories")
else:
    print(" No categories found to encode")


 Label Encoding Mappings:
   metal → 2
   glass → 1
   paper → 3
   cardboard → 0
   plastic → 4

 Label encoder created for 5 categories


In [8]:
# Function to process single image through complete pipeline
def preprocess_single_image(image_path, category, label_encoder, apply_augment=False):
    """
    Complete preprocessing pipeline for single image:
    1. Load image
    2. Resize to 256x256
    3. Apply augmentation (optional)
    4. Convert to array
    5. Normalize pixel values
    6. Encode label
    """
    # Step 1: Load image
    image = load_image(image_path)
    if image is None:
        return None, None
    
    # Step 2: Resize image to 256x256
    resized_image = resize_image(image, target_size=(256, 256))
    
    # Step 3: Apply augmentation if requested
    if apply_augment:
        processed_image = apply_augmentation(resized_image)
        # Ensure size is still 256x256 after augmentation
        processed_image = resize_image(processed_image, target_size=(256, 256))
    else:
        processed_image = resized_image
    
    # Step 4: Convert PIL image to numpy array
    image_array = image_to_array(processed_image)
    
    # Step 5: Normalize pixel values to 0-1 range
    normalized_array = normalize_image(image_array)
    
    # Step 6: Encode category label to numerical value
    encoded_label = encode_label(category, label_encoder)
    
    return normalized_array, encoded_label

# Function to process entire dataset
def process_complete_dataset(dataset_path, categories, label_encoder, augment_original=True):
    """Process complete dataset through preprocessing pipeline"""
    print(" Starting complete dataset preprocessing...")
    
    # Initialize lists to store processed data
    processed_images = []      # Store normalized image arrays
    processed_labels = []      # Store encoded labels
    processing_stats = {}      # Store processing statistics
    
    # Process each category
    for category in categories:
        print(f"\n Processing category: {category}")
        
        category_path = os.path.join(dataset_path, category)
        
        # Get all image files in category
        image_files = [f for f in os.listdir(category_path)
                      if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
        
        # Initialize category statistics
        original_count = len(image_files)
        processed_count = 0
        augmented_count = 0
        failed_count = 0
        
        # Process each image in category
        for filename in image_files:
            image_path = os.path.join(category_path, filename)
            
            # Process original image (without augmentation)
            img_array, label = preprocess_single_image(
                image_path, category, label_encoder, apply_augment=False
            )
            
            if img_array is not None:
                processed_images.append(img_array)
                processed_labels.append(label)
                processed_count += 1
                
                # Create augmented version if requested
                if augment_original:
                    aug_img_array, aug_label = preprocess_single_image(
                        image_path, category, label_encoder, apply_augment=True
                    )
                    
                    if aug_img_array is not None:
                        processed_images.append(aug_img_array)
                        processed_labels.append(aug_label)
                        augmented_count += 1
            else:
                failed_count += 1
        
        # Store category statistics
        processing_stats[category] = {
            'original': original_count,
            'processed': processed_count,
            'augmented': augmented_count,
            'failed': failed_count,
            'total_output': processed_count + augmented_count
        }
        
        print(f"    {category}: {original_count} → {processed_count + augmented_count} images")
        print(f"       Original: {processed_count}, Augmented: {augmented_count}, Failed: {failed_count}")
    
    # Convert lists to numpy arrays
    X_processed = np.array(processed_images)  # Image data
    y_processed = np.array(processed_labels)  # Labels
    
    print(f"\n Dataset preprocessing completed!")
    print(f" Final dataset shape: {X_processed.shape}")
    print(f" Labels shape: {y_processed.shape}")
    print(f" Total images: {len(X_processed)}")
    
    return X_processed, y_processed, processing_stats

print(" Complete preprocessing pipeline defined successfully")


 Complete preprocessing pipeline defined successfully


In [ ]:
# Execute the complete preprocessing pipeline
print(" Executing complete preprocessing pipeline...")
print("=" * 60)

# Process the dataset
X_data, y_data, stats = process_complete_dataset(
    dataset_path, categories, label_encoder, augment_original=True
)

# Display final statistics
print("\n" + "=" * 60)
print(" FINAL PROCESSING RESULTS:")
print("=" * 60)

total_original = sum([stats[cat]['original'] for cat in stats])
total_final = sum([stats[cat]['total_output'] for cat in stats])

print(f" Original images: {total_original}")
print(f" Final images: {total_final}")
print(f" Data augmentation ratio: {total_final/total_original:.2f}x")
print(f" Image shape: {X_data.shape[1:]}")
print(f" Data type: {X_data.dtype}")
print(f" Pixel value range: [{X_data.min():.3f}, {X_data.max():.3f}]")
print(f" Number of classes: {len(np.unique(y_data))}")

print("\n Label distribution:")
unique_labels, counts = np.unique(y_data, return_counts=True)
for label, count in zip(unique_labels, counts):
    category_name = decode_label(label, label_encoder)
    print(f"   {category_name} (Label {label}): {count} images")

print("\n Preprocessing pipeline completed successfully!")


 Executing complete preprocessing pipeline...
 Starting complete dataset preprocessing...

 Processing category: metal
    metal: 2000 → 4000 images
       Original: 2000, Augmented: 2000, Failed: 0

 Processing category: glass
    glass: 2000 → 4000 images
       Original: 2000, Augmented: 2000, Failed: 0

 Processing category: paper
    paper: 2000 → 4000 images
       Original: 2000, Augmented: 2000, Failed: 0

 Processing category: cardboard
    cardboard: 2000 → 4000 images
       Original: 2000, Augmented: 2000, Failed: 0

 Processing category: plastic
    plastic: 2000 → 4000 images
       Original: 2000, Augmented: 2000, Failed: 0
